# Gui Mapper Basic Prototype (FSE)

In [1]:
import pandas as pd
import json
from pandas.io.parsers import read_csv

In [2]:
fse_dataset = read_csv("prediction_widget_results.csv")
fse_dataset.loc[fse_dataset["filename"] == "etsy_0_signin"]

,app,filename,widget_id,prediction,correct,original GUI event id,original GUI event xpath,prediction canonical id,correct canonical id
16886,etsy,etsy_0_signin,0,NONE,NONE,NaN,NaN,NaN,NaN
16887,etsy,etsy_0_signin,1,NONE,NONE,NaN,NaN,NaN,NaN
16888,etsy,etsy_0_signin,8,NONE,NONE,NaN,NaN,NaN,NaN
16889,etsy,etsy_0_signin,9,NONE,NONE,NaN,NaN,NaN,NaN
16890,etsy,etsy_0_signin,10,NONE,NONE,NaN,NaN,NaN,NaN
16891,etsy,etsy_0_signin,11,NONE,NONE,NaN,NaN,NaN,NaN
16892,etsy,etsy_0_signin,12,email,email,com.etsy.android:id/edit_username,/hierarchy/android.widget.FrameLayout/android....,signin_username_or_email,signin_username_or_email
16893,etsy,etsy_0_signin,13,password,password,com.etsy.android:id/edit_password,/hierarchy/android.widget.FrameLayout/android....,signin_password,signin_password
16894,etsy,etsy_0_signin,14,NONE,NONE,NaN,NaN,NaN,NaN
16895,etsy,etsy_0_signin,15,signin_forgot,signin_forgot,com.etsy.android:id/forgot_password_link,/hierarchy/android.widget.FrameLayout/android....,signin_forget_pwd,signin_forget_pwd


In [3]:
etsy_csv = read_csv("../src/test_csv/Etsy.csv", names=["method", "json"])
etsy_csv

,method,json
0,<Etsy.WelcomeTest: void testSignIn()>,"[""{\""id_or_xpath\"":\""id@com.etsy.android:id/bt..."
1,<Etsy.WelcomeTest: void testSignIn()>,"[""{\""id_or_xpath\"":\""id@com.etsy.android:id/bt..."
2,<Etsy.WelcomeTest: void testSignIn()>,"[""{\""id_or_xpath\"":\""id@com.etsy.android:id/bt..."


In [4]:
def import_json(lst):
    lst = json.loads(lst)
    return [json.loads(str) for str in lst]

In [5]:
etsy_csv['json'] = etsy_csv['json'].apply(import_json)
etsy_csv

,method,json
0,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...
1,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...
2,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...


In [6]:
etsy_csv['json'][0]

[{'id_or_xpath': 'id@com.etsy.android:id/btn_link', 'action': 'click'},
 {'id_or_xpath': 'id@com.etsy.android:id/btn_link', 'action': 'click'},
 {'id_or_xpath': 'id@com.etsy.android:id/edit_username',
  'input': 'aaa',
  'action': 'sendKeys'},
 {'id_or_xpath': 'id@com.etsy.android:id/edit_password',
  'input': 'bbb',
  'action': 'sendKeys'},
 {'id_or_xpath': 'id@com.etsy.android:id/button_signin', 'action': 'click'}]

In [7]:
type(etsy_csv['json'][0][2])

dict

In [8]:
def gui_map(test):
    for gui_event in test:
        if gui_event['id_or_xpath'][:3] == "id@":
            search_col = "original GUI event id"
        else:
            search_col = "original GUI event xpath"
        search = fse_dataset.loc[fse_dataset[search_col] == gui_event['id_or_xpath'][3:]]
        
        if search.shape[0] == 0: #search returns nothing
            gui_event['case'] = "notFound"
        else:
            prediction = search.iloc[0]['prediction']
            correct = search.iloc[0]['correct']
            if prediction != "NONE":
                if prediction == correct:
                    gui_event['case'] = "correct"
                else:
                    gui_event['case'] = "incorrect"
            else:
                if correct != "NONE":
                    gui_event['case'] = "missed"
                else:
                    gui_event['case'] = "nonExist"
    return test

In [9]:
etsy_csv['json'] = etsy_csv['json'].apply(gui_map)
etsy_csv['json'][0]

[{'id_or_xpath': 'id@com.etsy.android:id/btn_link',
  'action': 'click',
  'case': 'notFound'},
 {'id_or_xpath': 'id@com.etsy.android:id/btn_link',
  'action': 'click',
  'case': 'notFound'},
 {'id_or_xpath': 'id@com.etsy.android:id/edit_username',
  'input': 'aaa',
  'action': 'sendKeys',
  'case': 'correct'},
 {'id_or_xpath': 'id@com.etsy.android:id/edit_password',
  'input': 'bbb',
  'action': 'sendKeys',
  'case': 'correct'},
 {'id_or_xpath': 'id@com.etsy.android:id/button_signin',
  'action': 'click',
  'case': 'correct'}]

Currently just adds another entry into the dictionary/json object to describe the case. This can be changed if necessary.

In [10]:
def list_cases(test):
    cases = {'correct' : [], 'incorrect' : [], 'missed' : [], 'nonExist' : [], 'notFound' : []}
    for gui_event in test:
        cases[gui_event['case']].append(gui_event['id_or_xpath'])
    return cases

In [11]:
output0 = pd.concat([etsy_csv, etsy_csv['json'].apply(list_cases).apply(pd.Series)], axis=1)
output0

,method,json,correct,incorrect,missed,nonExist,notFound
0,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...,"[id@com.etsy.android:id/edit_username, id@com....",[],[],[],"[id@com.etsy.android:id/btn_link, id@com.etsy...."
1,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...,"[id@com.etsy.android:id/edit_username, id@com....",[],[],[],"[id@com.etsy.android:id/btn_link, id@com.etsy...."
2,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...,"[id@com.etsy.android:id/edit_username, id@com....",[],[],[],"[id@com.etsy.android:id/btn_link, id@com.etsy...."


In [12]:
def count_cases(test):
    cases = {}
    cases['# correct'] = len(test['correct'])
    cases['# incorrect'] = len(test['incorrect'])
    cases['# missed'] = len(test['missed'])
    cases['# nonExist'] = len(test['nonExist'])
    cases['# notFound'] = len(test['notFound'])
    return cases

In [13]:
output = pd.concat([output0, output0.apply(count_cases, axis=1).apply(pd.Series)], axis=1)
output

,method,json,correct,incorrect,missed,nonExist,notFound,# correct,# incorrect,# missed,# nonExist,# notFound
0,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...,"[id@com.etsy.android:id/edit_username, id@com....",[],[],[],"[id@com.etsy.android:id/btn_link, id@com.etsy....",3,0,0,0,2
1,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...,"[id@com.etsy.android:id/edit_username, id@com....",[],[],[],"[id@com.etsy.android:id/btn_link, id@com.etsy....",3,0,0,0,2
2,<Etsy.WelcomeTest: void testSignIn()>,[{'id_or_xpath': 'id@com.etsy.android:id/btn_l...,"[id@com.etsy.android:id/edit_username, id@com....",[],[],[],"[id@com.etsy.android:id/btn_link, id@com.etsy....",3,0,0,0,2


Here I used the information to just count up the number of each case present in each test.